In [1]:
from pyspark.sql.functions import round, col, dayofmonth, month, year, to_date, quarter, substring, when, regexp_replace

StatementMeta(, 36359627-a89a-45e9-a435-c596455f64f8, 3, Finished, Available, Finished)

In [2]:
# Path to wind_power table in Bronze Lakehouse
bronze_table_path = "abfss://wind_turbine_energy@onelake.dfs.fabric.microsoft.com/LH_Bronze_WindEnergy.Lakehouse/Tables/dbo/wind_power"

# Load the wind_power table into a DataFrame
df = spark.read.format("delta").load(bronze_table_path)

StatementMeta(, 36359627-a89a-45e9-a435-c596455f64f8, 4, Finished, Available, Finished)

In [3]:
# Clean and enrich data
df_transformed = (df
    .withColumn("wind_speed", round(col("wind_speed"), 2))
    .withColumn("energy_produced", round(col("energy_produced"), 2))
    .withColumn("day", dayofmonth(col("date")))
    .withColumn("month", month(col("date")))
    .withColumn("quarter", quarter(col("date")))
    .withColumn("year", year(col("date")))
    .withColumn("time", regexp_replace(col("time"), "-", ":"))
    .withColumn("hour_of_day", substring(col("time"), 1, 2).cast("int"))
    .withColumn("minute_of_hour", substring(col("time"), 4, 2).cast("int"))
    .withColumn("second_of_minute", substring(col("time"), 7, 2).cast("int"))
    .withColumn("time_period", when((col("hour_of_day") >= 5) & (col("hour_of_day") < 12), "Morning")
                                .when((col("hour_of_day") >= 12) & (col("hour_of_day") < 17), "Afternoon")
                                .when((col("hour_of_day") >= 17) & (col("hour_of_day") < 21), "Evening")
                                .otherwise("Night")
    )
)

StatementMeta(, 36359627-a89a-45e9-a435-c596455f64f8, 5, Finished, Available, Finished)

In [4]:
# Path to wind_power table in Silver Lakehouse
silver_table_path = "abfss://wind_turbine_energy@onelake.dfs.fabric.microsoft.com/LH_Silver_WindEnergy.Lakehouse/Tables/dbo/wind_power"

StatementMeta(, 36359627-a89a-45e9-a435-c596455f64f8, 6, Finished, Available, Finished)

In [5]:
# Save the transformed table in the Silver Lakehouse
df_transformed.write.format("delta").mode("overwrite").save(silver_table_path)

StatementMeta(, 36359627-a89a-45e9-a435-c596455f64f8, 7, Finished, Available, Finished)